# MNIST Classification based on Tensorflow2.2 FrameWork

Statement:
MNIST dataset was processed as CSV format, pandas helps to download it into workspace, and it should be transfered to tensor format for least computation time.

Model is a four-layers Dense structure, whose units are designed as decreasing method.

The accuracy is 0.114 under 60000 samples for train. More information showed in tensorboard.

Requested:
* tensorflow 2.0 or last version
* pandas 0.25 or last version
* plotly 4.6 or last version

In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models, losses, callbacks
import plotly.express as px
import pandas as pd
import datetime

class Mnist:
    
    def __init__(self, file_name, index, epochs):
        self.file_name = file_name
        self.index = index
        self.epochs = epochs
        
        
    def load_file(self,file_name):
        data = pd.read_csv(self.file_name)

        train_x = tf.constant(data.drop(columns='class').iloc[0:60001].values)
        train_y = tf.constant(data['class'].iloc[0:6001].values)

        test_x = tf.constant(data.drop(columns='class').iloc[60001:].values)
        test_y = tf.constant(data['class'].iloc[60001:].values )

        return train_x, train_y, test_x, test_y

    def draw(self, data, index):

        fig=px.imshow(tf.reshape(data[self.index], [28, 28]))
        fig.show()
    #train_x, train_y, test_x, test_y = load_file('./Data/mnist/mnist_784.csv')
    #draw(train_x,0)

    def model1(self):
        tf.keras.backend.clear_session()

        model = models.Sequential()

        model.add(layers.Dense(40, activation='relu', input_shape=(784,)))
        model.add(layers.Dense(20, activation='relu'))
        model.add(layers.Dropout(rate=0.2))
        model.add(layers.Dense(10, activation='relu'))
        model.add(layers.Dense(1, activation='softmax'))

        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['Accuracy'])
        model.summary()

        return model 


    def model2(self):
        tf.keras.backend.clear_session()

        inputs = layers.Input(shape=(28,28))
        x = layers.Conv2D(28,kernel_size=(3,3))(inputs)
        x = layers.MaxPool2D()(x)
        x = layers.Conv2D(56,kernel_size=(5,5))(x)
        x = layers.MaxPool2D()(x)
        x = layers.Dropout(rate=0.1)(x)
        x = layers.Flatten()(x)
        x = layers.Dense(28,activation='relu')(x)
        outputs = layers.Dense(1,activation = 'sigmoid')(x)

        model = models.Model(inputs = inputs,outputs = outputs)

        model.summary()

        return model 

    def train(self, model, x, y,epochs):
        %load_ext tensorboard

        log_dir="./log/mnist1/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

        gpus = tf.config.list_physical_devices('GPU')
        print(gpus)
        with tf.device('gpu:0'):
            history=model.fit(x, y, batch_size=40, epochs=self.epochs, validation_split=0.2, 
                              callbacks=[tensorboard_callback])

        %tensorboard --logdir log/mnist1

        model.save_weights('./ModelTrained/mist1.cpkt', save_format = 'tf')
        model.save('./ModelTrained/mist1', save_format='tf')

        return model 

    def model_evaluate(self, model_trained, test_x, test_y):
        loss_value, metrics_value = model_trained.evaluate(test_x, test_y)
        print('loss_value:', loss_value, 'metrics_value:', metrics_value)

        return loss_value, metrics_value 

In [5]:
mnist = Mnist('./Data/mnist/mnist_784.csv', 0, 30)
train_x, train_y, test_x, test_y = mnist.load_file('./Data/mnist/mnist_784.csv')
mnist.draw(train_x,0)

In [6]:
model=mnist.model1()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 40)                31400     
_________________________________________________________________
dense_1 (Dense)              (None, 20)                820       
_________________________________________________________________
dropout (Dropout)            (None, 20)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 32,441
Trainable params: 32,441
Non-trainable params: 0
_________________________________________________________________


In [3]:
model=mnist.train(model, train_x, train_y,30)

NameError: name 'model' is not defined

In [97]:
mnist.model_evaluate(model,test_x, test_y)

2000/2000 [==============================] - 3s 2ms/step - loss: 0.0000e+00 - accuracy: 0.1126
loss_value: 0.0 metrics_value: 0.11259550601243973


(0.0, 0.11259550601243973)